<a href="https://colab.research.google.com/github/rodrigofer89/cybersecurity/blob/main/Global_Solution_CyberSecurity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Apenas para evitar warnings 
import warnings
warnings.filterwarnings('ignore')

# Bibliotecas de Data Science
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from matplotlib import pyplot

# Padronização dos dados
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

# Modelo a ser utilizado
from sklearn.model_selection import train_test_split
from sklearn import linear_model

# Métricas de avaliação
from sklearn.metrics import classification_report, accuracy_score


In [ ]:
df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

In [ ]:
df.shape

(69858, 11)

In [ ]:
df.head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0.0,0.0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0.0,0.0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1.0,0.0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1.0,0.0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69858 entries, 0 to 69857
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   step            69858 non-null  int64  
 1   type            69858 non-null  object 
 2   amount          69858 non-null  float64
 3   nameOrig        69858 non-null  object 
 4   oldbalanceOrg   69858 non-null  float64
 5   newbalanceOrig  69858 non-null  float64
 6   nameDest        69858 non-null  object 
 7   oldbalanceDest  69858 non-null  float64
 8   newbalanceDest  69857 non-null  float64
 9   isFraud         69857 non-null  float64
 10  isFlaggedFraud  69857 non-null  float64
dtypes: float64(7), int64(1), object(3)
memory usage: 5.9+ MB


In [ ]:
df.isna().sum()

NameError: ignored

### *Não temos valores nulos*

In [ ]:
df.describe()

### Dando uma olhada na coluna target conseguimos ver que temos poucos exemplos de fraude, isso pode ser um problema

In [ ]:
# Correlação
upp_mat = np.triu(df.corr())
sb.heatmap(df.corr()[['isFraud']].sort_values(by='isFraud', ascending=False), vmin = - 1, vmax = + 1, annot = True, cmap = 'coolwarm')

In [ ]:
# Histograma das colunas númericas
for col in df.columns: 
    if df.dtypes[col] in ('int64','float64'):
        fig = plt.figure(figsize =(3, 3))
        plt.title(col)
        pyplot.hist(df[col])
        pyplot.show()

In [ ]:
# Teste de Shapiro
variavel_normal = 0
variavel_nao_normal = 0

from scipy.stats import shapiro
for col in df.columns: 
    if df.dtypes[col] in ('int64','float64'):
        stat, p = shapiro(df[col])
        print(col)
        print('Statistics=%.3f, p=%.23f' % (stat, p))
        alpha = 0.05
        if p > alpha:
            print('Distribuição Normal')
            variavel_normal += 1
        else:
            print('Distribuição não normal')
            variavel_nao_normal += 1
        print("----------------------------")
quantidade_variaveis = variavel_normal + variavel_nao_normal
resultado_shapiro,distribuicao_normal = ("seguem uma distribuição normal",1) \
                                        if variavel_normal > variavel_nao_normal \
                                        else ("não seguem uma distribuição normal",0)
print(f'Com base na análise das {quantidade_variaveis} variáveis numéricas através do teste de Shapiro, \n' 
      f'foi possível verificar que a maioria das variáveis {resultado_shapiro}. \n'
      f'Quantidade variáveis que seguem uma distribuição normal:{variavel_normal} \n'
      f'Quantidade variáveis que não seguem uma distribuição normal:{variavel_nao_normal} \n')

*   Analise de correlação mostrou que a coluna AMOUNT tem uma alta correlação positiva com a coluna ISFRAUD, já as que envolvem o BALANCE tem uma alta correlação negativa

* Foi feito um histograma das colunas númericas para verificar a distribuição dos dados, com isso vimos que os dados não seguem uma distribuição normal

* Com o teste de Shapiro conseguimos ver que realmente o dataset não segue uma distribuição normal



In [ ]:
# plot da coluna 'type'
fig, ax = plt.subplots()
sb.countplot(df.isFraud,hue=df.type,ax=ax)
for i in ax.patches:
  valor = i.get_x() + i.get_height()
  x = i.get_x() + i.get_width()
  y = i.get_y() + i.get_height()
  ax.annotate(valor,(x,y))


In [ ]:
# Observando quais transações tem ocorrência de fraude
print('Tipos de transações Fraudulentos: ', list(df.loc[(df.isFraud == 1)].type.unique()))

In [ ]:
# Vendo a distribuição dos dados que tem ocorrência de fraude
print(df['isFraud'].value_counts())
print('=-' *15)
print(df['isFraud'].value_counts(normalize=True))

*   Decidimos olhar a coluna TYPE para observar quais transações tem maior indice de fraude, com isso conseguimos ver que apenas dois tipos de transações nesse exemplo são sinalizados com fraude, TRANSFER e CASH_OUT

*   Olhando somente os valores de fraude ou não, temos uma melhor visão de que o dataset está muito desbalanciado cerca de 99% dos dados não são fraudes

In [ ]:
df['isFlaggedFraud'].value_counts()

In [ ]:
len(df.loc[(df['isFlaggedFraud'] == 1)& (df.isFraud == 1)])

*   Olhando a coluna ISFRAGGEDFRAUD conseguimos ver que existiam 16 sinalizações de possiveis fraudes então comparamos com a coluna ISFRAUD e vimos que todas as sinalizações em ISFRAGGEDFRAUD estão presentes em ISFRAUD então é uma coluna que não parece útil.

### ***Data Cleaning*** and ***Feature Engineering***

In [ ]:
df.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1, inplace = True)

In [ ]:
df2 = df.loc[(df.type == 'TRANSFER')|(df.type == 'CASH_OUT')]

In [ ]:
df2['type'].unique()

In [ ]:
df2['type'] = LabelEncoder().fit_transform(df2['type'])

*   Agora retiramos as colunas que após a analise não fazem sentido

*   Como já tinhamos visto as transações com ocorrência de fraude são apenas 'TRANSFER' e 'CASH_OUT' então tratamos a coluna type deixando apenas as transações com ocorrências.

*   Com o LabelEncoder tratamos a coluna type e colocamos os valores entre 0 e 1 lembrando que essa era nossa unica coluna texto.






### ***Modelling***

In [ ]:
# Separando a coluna target em 'y' e o restante do dataset para normalizar em 'X'
X = df2.drop(columns=['isFraud'])
y = df2['isFraud']

In [ ]:
# Normalização
X_norm = MinMaxScaler().fit_transform(X)
X_norm

In [ ]:
# Dividindo o dataset entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_norm,
                                                    y,
                                                    test_size = .30,
                                                    random_state=42)

In [ ]:
# Modelo de regressão logistica
logitic = linear_model.LogisticRegression()
logitic.fit(X_train,y_train)
y_pred = logitic.predict(X_test)
print(classification_report(y_test,y_pred))
accuracy_score(y_test,y_pred)

### Conclusão:

*   Após a analise nosso modelo teve uma precisão de 98% estamos levando em conta somente a precisão pois nesse caso o falso positivo pode ser mais prejudicial financeiramente, foram feitos vários testes e nosso melhor resultado foi esse, usamos também o normalizador ‘StandarScaler’ e tivemos um resultado de 90% de precisão com isso escolhemos o ‘MinMaxScaler’.

*   O problema parece a quantidade de ocorrências de fraude o que dificulta fazendo com que o dataset seja desequilibrado, acreditamos que com mais exemplos e melhor tratamento dos dados nos conseguiremos melhorar as métricas.

